In [48]:
import dotenv
import os

In [36]:
project_dir = os.path.join(os.path.dirname('./src/'), os.pardir)

In [37]:
 os.pardir

'..'

In [40]:
project_dir

'./src\\..'

In [39]:
dotenv_path = os.path.join(project_dir, '.env')

In [23]:
dotenv_path

'/..\\.env'

In [49]:
import sys

In [50]:
sys.path

['C:\\Users\\xauspi\\Documents\\git_repository\\adv_dsi\\kaggle_assignment',
 'C:\\Users\\xauspi\\Documents\\git_repository\\adv_dsi\\kaggle_assignment\\src',
 'C:\\Users\\xauspi\\AppData\\Local\\Programs\\Python\\Python39\\python39.zip',
 'C:\\Users\\xauspi\\AppData\\Local\\Programs\\Python\\Python39\\DLLs',
 'C:\\Users\\xauspi\\AppData\\Local\\Programs\\Python\\Python39\\lib',
 'C:\\Users\\xauspi\\AppData\\Local\\Programs\\Python\\Python39',
 'c:\\users\\xauspi\\.virtualenvs\\kaggle_assignment-dq2gjalz',
 '',
 'c:\\users\\xauspi\\.virtualenvs\\kaggle_assignment-dq2gjalz\\lib\\site-packages',
 'c:\\users\\xauspi\\.virtualenvs\\kaggle_assignment-dq2gjalz\\lib\\site-packages\\win32',
 'c:\\users\\xauspi\\.virtualenvs\\kaggle_assignment-dq2gjalz\\lib\\site-packages\\win32\\lib',
 'c:\\users\\xauspi\\.virtualenvs\\kaggle_assignment-dq2gjalz\\lib\\site-packages\\Pythonwin',
 'c:\\users\\xauspi\\.virtualenvs\\kaggle_assignment-dq2gjalz\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\

In [43]:
from src.models import eval_model